


<font size='10' color = 'E3A440'>**Mégadonnées et techniques avancées démystifiées**</font>
=======
<font color = 'E3A440'>*Nouvelles méthodes d’analyse et leur implication quant à la gestion des mégadonnées en SSH (partie 1)*</font>
=============


Cet atelier s’inscrit dans le cadre de la formation [Mégadonnées et techniques avancées démystifiées](https://www.4point0.ca/2022/08/22/formation-megadonnees-demystifiees/) (séance 6).

Les sciences humaines et sociales sont souvent confrontées à l’analyse de données non structurées, comme le texte. Après avoir préparé les données, plusieurs techniques d’analyse venant de l’apprentissage automatique peuvent être utilisées. Pendant cet atelier, les participants seront initiés aux méthodes supervisées et non supervisées à des buts d’analyse avec Python.

Note : Cet atelier se poursuit lors d’une 2e séance le 10 novembre.

Structure de l'atelier :
1. Presentation of sections 1 and 2 in a plenary mode (20 minutes)
2. Individual work on section 3 (20 minutes)
3. Group work on section 4 (60 minutes)
4. Plenary session with groups presentations (20 minutes)

Ce tutoriel ne peut pas être consideré exaustif .... 

### Authors: 
- Bruno Agard <bruno.agard@polymtl.ca>
- Davide Pulizzotto <davide.pulizzotto@polymtl.ca>

### Table of Contents
Bruno Agard

Département de Mathématiques et de génie Industriel

École Polytechnique de Montréal

# Préparation environnement

In [116]:
# Downloading of data from the GitHub project
!rm -rf Donnees_demystifiees_seance_6/
!git clone https://github.com/puli83/Donnees_demystifiees_seance_6

Cloning into 'Donnees_demystifiees_seance_6'...
fatal: could not read Username for 'https://github.com': No such device or address


In [3]:
# Import modules
import nltk
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

# <font color = 'E3A440'>*Préparation des données textulles*</font>

L'analyse de données textuelles implique la transformation d'un texte en un objet mathematique qui peut être utilisé par des algorithmes et des modèles statistique. Cette étape est importante car permet de **structurer** des données non structurées, comme le texte.


###  <font color = 'E3A440'>**Étapes fondamentales du prétraitement**</font>

Prenons une phrase pour decortiquer les passages que nous pemrettent de la transformer en information structurée.

In [3]:
sentence = """At eight o'clock, on Thursday morning, the great Arthur didn't feel VERY good."""

In [4]:
len(sentence)

78

#### <font color = 'E3A440'>*1. Tokenisation*</font>

Cette étape consiste à couper la phrase en unités linguistiques élémantaire et dotées de sens, ce qui est gnralement appelé le "mot".

Dans le module `nltk`, il existe une fonction qui permet cette opération, soit `word_tokenize()`.

In [5]:
# La function word_tokenize() prend la phrase comme argument.
words = nltk.word_tokenize(sentence)
print(words)
len(words)

['At', 'eight', "o'clock", ',', 'on', 'Thursday', 'morning', ',', 'the', 'great', 'Arthur', 'did', "n't", 'feel', 'VERY', 'good', '.']


17

#### <font color = 'E3A440'>*2. Analyse morphosyntaxique*</font>

Après avoir identifé tous les mots, il est possible de analyser leur rôle morphosyntaxique, à des fins d'analyse et/ou filtrage. 

In [6]:
 # La function word_tokenize() prend la liste de mots comme argument.
words_pos = nltk.pos_tag(words, tagset='universal')
print(words_pos)
len(words_pos)

[('At', 'ADP'), ('eight', 'NUM'), ("o'clock", 'NOUN'), (',', '.'), ('on', 'ADP'), ('Thursday', 'NOUN'), ('morning', 'NOUN'), (',', '.'), ('the', 'DET'), ('great', 'ADJ'), ('Arthur', 'NOUN'), ('did', 'VERB'), ("n't", 'ADV'), ('feel', 'VERB'), ('VERY', 'ADV'), ('good', 'ADJ'), ('.', '.')]


17

Voici la liste de possible POS tags:

| **POS** | **DESCRIPTION**           | **EXAMPLES**                                      |
| ------- | ------------------------- | ------------------------------------------------- |
| ADJ     | adjective                 | big, old, green, incomprehensible, first      |
| ADP     | adposition                | in, to, during                                |
| ADV     | adverb                    | very, tomorrow, down, where, there            |
| AUX     | auxiliary                 | is, has (done), will (do), should (do)        |
| CONJ    | conjunction               | and, or, but                                  |
| CCONJ   | coordinating conjunction  | and, or, but                                  |
| DET     | determiner                | a, an, the                                    |
| INTJ    | interjection              | psst, ouch, bravo, hello                      |
| NOUN    | noun                      | girl, cat, tree, air, beauty                  |
| NUM     | numeral                   | 1, 2017, one, seventy-seven, IV, MMXIV        |
| PART    | particle                  | ’s, not                                      |
| PRON    | pronoun                   | I, you, he, she, myself, themselves, somebody |
| PROPN   | proper noun               | Mary, John, London, NATO, HBO                 |
| PUNCT   | punctuation               | ., (, ), ?                                    |
| SCONJ   | subordinating conjunction | if, while, that                               |
| SYM     | symbol                    | $, %, §, ©, +, −, ×, ÷, =, :)               |
| VERB    | verb                      | run, runs, running, eat, ate, eating          |
| X       | other                     | sfpksdpsxmsa                                  |
| SPACE   | space                     |                                                   |


#### <font color = 'E3A440'>*3. Retirer la ponctuation*</font>

Une autre opération consiste à retirer la ponctuation. Ce type de filtrage reduit le nombre de sugne graphique qui participent le moin à la constructuion de la sémantique de la prhase. 
Dans certain contexte, comme en stylometrie, ce processus est appliquée avec de terchniques plus sofistiquées. 

In [7]:
# La ligne de code suivant itére sur chaque signe graphique et retient ceux qui contiennet de caracteres alphanumérique.
words_pos = [(w, pos) for w, pos in words_pos if w.isalnum()]
print(words_pos)
len(words_pos)
# Ikl est possible aussi d'utiilser le résultat de l'analyse morphosyntaxique pour éliminer la ponctuaction
# words_pos = [(w, pos) for w, pos in words_pos if pos != '.']
# print(words_pos)

[('At', 'ADP'), ('eight', 'NUM'), ('on', 'ADP'), ('Thursday', 'NOUN'), ('morning', 'NOUN'), ('the', 'DET'), ('great', 'ADJ'), ('Arthur', 'NOUN'), ('did', 'VERB'), ('feel', 'VERB'), ('VERY', 'ADV'), ('good', 'ADJ')]


12

#### <font color = 'E3A440'>*4. Convertir chaque caractère en minuscule*</font>

Cette étape constitue une première opéraiton de normalisation des mots et leur réduction à une forme graphique unique. Ce genre d'étape permet de regrouper chaque occurence d'un mot sous une seule forme.

In [8]:
# La ligne de code suivant itére sur chaque signe graphique et le transforme en minuscule.
words_pos = [(w.lower(), pos) for w, pos in words_pos]
print(words_pos)

[('at', 'ADP'), ('eight', 'NUM'), ('on', 'ADP'), ('thursday', 'NOUN'), ('morning', 'NOUN'), ('the', 'DET'), ('great', 'ADJ'), ('arthur', 'NOUN'), ('did', 'VERB'), ('feel', 'VERB'), ('very', 'ADV'), ('good', 'ADJ')]


#### <font color = 'E3A440'>*5. retirer les stopwrods (mots vides)*</font>

Une autre opération de filtrage constitue dan l'élimination de mots fonctionnels. Cette liste de mots contient tout les connecteurs de phrase, comme "et", "mais", "toutefois" et de mots avec faible valeur sémantique, comem les verbes modaux. 
Comme d'autres opéraiton de filtrage, l'enjeuy est celui de nettoyer le plus possible le vocabulaire et de reduyire toutes les occurrences d'un mot sous une forme graphique unique.

In [9]:
# Nous importons la liste de stopword en anglais
from nltk.corpus import stopwords
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [10]:
# La ligne de code suivant itére sur chaque signe graphique et garde ce qui n sont pas dans la liste de stopword.
words_pos = [(w, pos) for w, pos in words_pos if w not in stopwords.words("english")]
print(words_pos)
len(words_pos)

[('eight', 'NUM'), ('thursday', 'NOUN'), ('morning', 'NOUN'), ('great', 'ADJ'), ('arthur', 'NOUN'), ('feel', 'VERB'), ('good', 'ADJ')]


7

#### <font color = 'E3A440'>*6. Rammener les mots à leur racine*</font> 

En suivant le même objectif, nous retirons le suffixe morphologique des mots, ce qui augmente le niveau de réduction de chaque occurrence d'un mot à une unique forme graphique.

Ils existent deux méthode fondamentales: la racinisaiton et la lemmatisation.
La première reduit les occurence à une racine qui est inférée au moyen de plusieur techniques, l'autre est la réduciton de l'occurrence à son lemme. 

In [11]:
# Racinisation: technique Porter
from nltk.stem.porter import PorterStemmer
stemmed_pos = [(PorterStemmer().stem(w), pos) for w, pos in words_pos]
print(stemmed_pos)

[('eight', 'NUM'), ('thursday', 'NOUN'), ('morn', 'NOUN'), ('great', 'ADJ'), ('arthur', 'NOUN'), ('feel', 'VERB'), ('good', 'ADJ')]


In [12]:
# Racinisation: technique Lancaster
from nltk.stem import LancasterStemmer
stemmed_pos = [(LancasterStemmer().stem(w), pos) for w, pos in words_pos]
print(stemmed_pos)

[('eight', 'NUM'), ('thursday', 'NOUN'), ('morn', 'NOUN'), ('gre', 'ADJ'), ('arth', 'NOUN'), ('feel', 'VERB'), ('good', 'ADJ')]


In [13]:
# Lemmatisaiton: utilisant le thesaurus wordnet
from nltk.stem.wordnet import WordNetLemmatizer
lemmed_pos = [(WordNetLemmatizer().lemmatize(w), pos) for w, pos in words_pos]
print(lemmed_pos)

[('eight', 'NUM'), ('thursday', 'NOUN'), ('morning', 'NOUN'), ('great', 'ADJ'), ('arthur', 'NOUN'), ('feel', 'VERB'), ('good', 'ADJ')]


#### <font color = 'E3A440'>*7. Filtrage selon le rôle morphosyntaxique*</font>

Le filtrage des unités lexicales peut s'étendre jusqu'à l'élimination d'unités qui ne font pas partie d'une liste de rôles morphosyntaxique prédéfinie. 

In [14]:
# Retenir seulement les noms et les adjectifs
lemmed_pos = [(w, pos) for w, pos in words_pos if pos in ['NOUN','ADJ']]
print(lemmed_pos)

[('thursday', 'NOUN'), ('morning', 'NOUN'), ('great', 'ADJ'), ('arthur', 'NOUN'), ('good', 'ADJ')]


## <font color = 'E3A440'>Traitement d'un texte</font>

Le prétraitement d'un corpus de recherche peut mettre en place plusieurs autres étapes. La plus importante est la **segmentation**. 

### <font color = 'E3A440'>*1. Segmentation*</font>

Tout dépendant de l'objectif de l'analyse, un segment peut être un document, un paragraphe, une concordance, un groupe de phrases, une phrase, etc.



In [15]:
text = """At eight o'clock, on Thursday morning, the great Arthur didn't feel VERY good.
          The following morning, at nine, Arthur felt better.
          A dog run in the street."""
len(text)

175

Dans le bloc de code suivant, nous faisons une segmentation par pharse.

In [34]:
sentences = nltk.sent_tokenize(text)
print(sentences)
len(sentences)

["At eight o'clock, on Thursday morning, the great Arthur didn't feel VERY good.", 'The following morning, at nine, Arthur felt better.', 'A dog run in the street.']


3

### <font color = 'E3A440'>*2. Annotation et nettoyage*</font>

Tout dépendant de l'objectif de l'analyse, un segment peut être un document, un paragraphe, une concordance, un groupe de phrases, une phrase, etc.


#### <font color = 'E3A440'>*2.1 Création d'une fonction*</font>

La cération de funciton est utile pour plusieurs raison. Dans notre casm, nous voulons englober Souvent, il est utile de créer de fonctions pour executer accomplir pluseurs étapes

In [35]:
# To run this function proprlely, you need to import modules needed
def CleaningText(text_as_string, language = 'english', reduce = '', list_pos_to_keep = [], Stopwords_to_add = []):
    from nltk.corpus import stopwords

    words = nltk.word_tokenize(text_as_string)
    words_pos = nltk.pos_tag(words, tagset='universal')
    words_pos = [(w, pos) for w, pos in words_pos if w.isalnum()]
    words_pos = [(w.lower(), pos) for w, pos in words_pos]
    
    if reduce == 'stem': 
        from nltk.stem.porter import PorterStemmer
        reduced_words_pos = [(PorterStemmer().stem(w), pos) for w, pos in words_pos]
        
    elif reduce == 'lemma':
        from nltk.stem.wordnet import WordNetLemmatizer
        reduced_words_pos = [(WordNetLemmatizer().lemmatize(w), pos) for w, pos in words_pos]
    else:
        import warnings
        reduced_words_pos = words_pos
        warnings.warn("Warning : any reduction was made on words! Please, use \"reduce\" argument to chosse between 'stem' or  'lemma'")
    if list_pos_to_keep:
        reduced_words_pos = [(w, pos) for w, pos in reduced_words_pos if pos in list_pos_to_keep]
    else:
        import warnings
        warnings.warn("Warning : any POS filtering was made. Pleae, use \"list_pos_to_keep\" to create a list of POS tag to keep.")
    
    list_stopwords = stopwords.words(language) + Stopwords_to_add
    reduced_words_pos = [(w, pos) for w, pos in reduced_words_pos if w not in list_stopwords and len(w) > 1 ]
    return reduced_words_pos



In [145]:
words = nltk.word_tokenize(text_as_string)
words_pos = nltk.pos_tag(words, tagset='universal')
words_pos = [(w, pos) for w, pos in words_pos if w.isalnum()]
words_pos = [(w.lower(), pos) for w, pos in words_pos]
from nltk.stem.wordnet import WordNetLemmatizer
reduced_words_pos = [(WordNetLemmatizer().lemmatize(w), pos) for w, pos in words_pos]
reduced_words_pos = [(w, pos) for w, pos in reduced_words_pos if pos in ['NOUN','ADJ','VERB']]
list_stopwords = stopwords.words(language) + ['http']
words_pos = [(w, pos) for w, pos in reduced_words_pos if w not in list_stopwords ]
reduced_words_pos



[('entire', 'ADJ'),
 ('swiss', 'ADJ'),
 ('football', 'NOUN'),
 ('league', 'NOUN'),
 ('is', 'VERB'),
 ('hold', 'NOUN'),
 ('postponing', 'VERB'),
 ('game', 'NOUN'),
 ('professional', 'ADJ'),
 ('amateur', 'ADJ'),
 ('level', 'NOUN'),
 ('coronavirus', 'ADJ'),
 ('http', 'NOUN')]

#### <font color = 'E3A440'>*2.2 Application function nettoyage*</font>

Maintenant, nous pouvons apliquer cette function à chacun de not segment. dans notre cas il s'agit de phrases.

In [37]:
cleaned_sentences = [CleaningText(sent) for sent in sentences]
print(cleaned_sentences)

[[('eight', 'NUM'), ('thursday', 'NOUN'), ('morning', 'NOUN'), ('great', 'ADJ'), ('arthur', 'NOUN'), ('feel', 'VERB'), ('good', 'ADJ')], [('following', 'ADJ'), ('morning', 'NOUN'), ('nine', 'NUM'), ('arthur', 'NOUN'), ('felt', 'VERB'), ('better', 'ADV')], [('dog', 'NOUN'), ('run', 'NOUN'), ('street', 'NOUN')]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: Warning : any reduction was made on words! Please, use "reduce" argument to chosse between 'stem' or  'lemma'
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: Warning : any POS filtering was made. Pleae, use "list_pos_to_keep" to create a list of POS tag to keep.


In [ ]:
cleaned_sentences = [CleaningText(sent, reduce = 'lemma', list_pos_to_keep = ['']) for sent in sentences]
print(cleaned_sentences)

#### <font color = 'E3A440'>*2.3 Fréquence mots*</font>

Quelle est la fréquence des mots de notre texte? Commencon par retirer le POS tag, pour obtenir une liste de listes de mots ( et non une liste de tuple mot-pos).

In [48]:
freqs_in_text = nltk.FreqDist([w for sent in cleaned_sentences for w, pos in sent ])
freqs_in_text

FreqDist({'morning': 2, 'arthur': 2, 'eight': 1, 'thursday': 1, 'great': 1, 'feel': 1, 'good': 1, 'following': 1, 'nine': 1, 'felt': 1, ...})

### <font color = 'E3A440'>*3. Vectorisation*</font>

Généralement, pour utiliser le texte dans un contexte d'analyse de données ou d'apprentissage automatique, ce texte doit être transformé dans un objet mathématique approprié. 
Le modèle le plus simple et connu est le "bags-of-words", dans lequel chaque document (ou chaque mot) est défini par un certain nombre d'unités lexicales qui le caractérise. 

In [38]:
# Initialisation de l'objet
from nltk.corpus import stopwords

def identity_tokenizer(text):
    return text

# Transforming the word in frequencies
vectorized = CountVectorizer(lowercase = False, # Convert all characters to lowercase before tokenizing
                             min_df = 1, # Ignore terms that have a document frequency strictly lower than the given threshold 
                             max_df = 10, # Ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words)
                             stop_words = stopwords.words('english'), # Remove the list of words provided
                             ngram_range = (1, 1), # Get the lower and upper boundary of the range of n-values for different word n-grams or char n-grams to be extracted
                             tokenizer=identity_tokenizer) # Override the string tokenization step while preserving the preprocessing and n-grams generation steps

Utilisation du "vectorizer" avec une liste de listes de mot (et non une liste de tuple de mots-pos).

In [49]:
# Liste de liste de mots:
[[w for w, pos in sent] for sent in cleaned_sentences]

[['eight', 'thursday', 'morning', 'great', 'arthur', 'feel', 'good'],
 ['following', 'morning', 'nine', 'arthur', 'felt', 'better'],
 ['dog', 'run', 'street']]

In [79]:
# pplication du vectorizer
freq_term_DTM = vectorized.fit_transform([[w for w, pos in sent] for sent in cleaned_sentences])
print(pd.DataFrame(freq_term_DTM.todense(), columns =  [k for k, v in sorted(vectorized.vocabulary_.items(), key=lambda item: item[1])] ))

   arthur  better  dog  eight  feel  felt  following  good  great  morning  \
0       1       0    0      1     1     0          0     1      1        1   
1       1       1    0      0     0     1          1     0      0        1   
2       0       0    1      0     0     0          0     0      0        0   

   nine  run  street  thursday  
0     0    0       0         1  
1     1    0       0         0  
2     0    1       1         0  


## Exercise : Sentiment analysis on a COVID-19 twetter dataset

In [118]:
ROOT_DIR='Donnees_demystifiees_seance_6/'
DATA_DIR=os.path.join(ROOT_DIR, 'Data')
os.listdir(DATA_DIR)

FileNotFoundError: ignored

In [117]:
!pwd

/content


In [108]:
#import gzip
import zipfile
!pip install wget
import wget
from datetime import datetime

#dataset_URL = "http://www.trackmyhashtag.com/data/COVID-19.zip" #@param {type:"string"}
dataset_URL = "https://drive.google.com/drive/folders/11w4geFB6p17hFlWseBpHJQbhARINvTOc" #@param {type:"string"}
#Downloads the dataset
wget.download(dataset_URL, out='Top_50_profile-test.zip')
#Unzips the dataset and gets the TSV dataset
with zipfile.ZipFile('Top_50_profile-test.zip', 'r') as zip_ref:
    zip_ref.extractall()


#dateparse = lambda x: datetime.strptime(x, '%d %b %Y %H:%M:%S')
#df = pd.read_csv(infile, parse_dates=['datetime'], date_parser=dateparse)
#Import dataset
#df = pd.read_csv('COVID.csv', parse_dates=['Tweet Posted Time (UTC)', 'User Account Creation Date'], date_parser=dateparse)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


BadZipFile: ignored

In [106]:
import os
os.listdir()

['.config',
 'COVID-19 (1).zip',
 'COVID-images.csv',
 'Top_50_profile.zip',
 'Top_50_profile (1).zip',
 'COVID-videos.csv',
 'COVID.csv',
 'COVID-19.zip',
 'sample_data']

In [ ]:
import datetime
#Select english
df=df[df['Tweet Language']=='English'].reset_index()
df

In [110]:
df.dtypes

index                                  int64
Tweet Id                              object
Tweet URL                             object
Tweet Posted Time (UTC)       datetime64[ns]
Tweet Content                         object
Tweet Type                            object
Client                                object
Retweets Received                      int64
Likes Received                         int64
Tweet Location                        object
Lat                                  float64
Long                                 float64
Tweet Language                        object
User Id                               object
Name                                  object
Screen Name                           object
User Bio                              object
Verified or Non-Verified              object
Profile URL                           object
Protected or Non-protected            object
User Followers                         int64
User Following                         int64
User Accou

In [109]:
df.iloc[0]

index                                                                         0
Tweet Id                                                  "1233417783175778304"
Tweet URL                     https://twitter.com/Giussi92/status/1233417783...
Tweet Posted Time (UTC)                                     2020-02-28 15:44:49
Tweet Content                 Also the entire Swiss Football League is on ho...
Tweet Type                                                                Tweet
Client                                                       Twitter for iPhone
Retweets Received                                                             0
Likes Received                                                                0
Tweet Location                                                              NaN
Lat                                                                         NaN
Long                                                                        NaN
Tweet Language                          

In [106]:
df.shape

(33174, 22)

In [121]:
df['Tweet Content']

0        Also the entire Swiss Football League is on ho...
1        World Health Org Official: Trump’s press confe...
2        I mean, Liberals are cheer-leading this #Coron...
3        Under repeated questioning, Pompeo refuses to ...
4        #coronavirus comments now from @larry_kudlow h...
                               ...                        
33169    RT @timhquotes: It's my party, you're invited!...
33170    RT @timhquotes: It's my party, you're invited!...
33171    It's my party, you're invited!\n\nPS, this is ...
33172    Amy’s a survivor! #bariclab #pnnl #movingon #c...
33173    A review of asymptomatic and sub-clinical Midd...
Name: Tweet Content, Length: 33174, dtype: object

Cette opération prendra 2 minutes, donc, nous vous suggérons de continuer la lecture dce l'exercices pour vous faire une idée de ce qui s'en vient.

In [36]:
cleaned_tweets = [CleaningText(sent, reduce = 'lemma', list_pos_to_keep = ['NOUN', 'ADJ', 'VERB'], Stopwords_to_add=['http']) for sent in list(df['Tweet Content'])]


In [73]:
# Initialisation de l'objet
from nltk.corpus import stopwords

def identity_tokenizer(text):
    return text

# Transforming the word in frequencies
vectorized = CountVectorizer(lowercase = False, # Convert all characters to lowercase before tokenizing
                             min_df = 25, # Ignore terms that have a document frequency strictly lower than the given threshold 
                             max_df = 25000, # Ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words)
                             stop_words = stopwords.words('english'), # Remove the list of words provided
                             ngram_range = (1, 1), # Get the lower and upper boundary of the range of n-values for different word n-grams or char n-grams to be extracted
                             tokenizer=identity_tokenizer) # Override the string tokenization step while preserving the preprocessing and n-grams generation steps

In [74]:
freq_term_DTM = vectorized.fit_transform([[w for w, pos in sent] for sent in cleaned_tweets])

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'", 'b', 'c', 'e', 'f', 'g', 'h', 'j', 'l', 'n', 'p', 'r', 'u', 'v', 'w'] not in stop_words.
  % sorted(inconsistent)


In [75]:
freq_term_DTM

<33174x2352 sparse matrix of type '<class 'numpy.int64'>'
	with 419189 stored elements in Compressed Sparse Row format>

In [150]:
df['Tweet Content'].iloc[[0,1,2,3]]

0    Also the entire Swiss Football League is on ho...
1    World Health Org Official: Trump’s press confe...
2    I mean, Liberals are cheer-leading this #Coron...
3    Under repeated questioning, Pompeo refuses to ...
Name: Tweet Content, dtype: object

In [147]:
print(CleaningText(df['Tweet Content'].iloc[0], reduce = 'lemma', list_pos_to_keep = ['NOUN', 'ADJ', 'VERB'], Stopwords_to_add=['http']))

text_as_string = df['Tweet Content'].iloc[0]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [6]:
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()
sia.polarity_scores("Wow, NLTK is really powerful!")

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


{'neg': 0.0, 'neu': 0.295, 'pos': 0.705, 'compound': 0.8012}

In [7]:
sia.polarity_scores("NLTK is not bad!")

{'neg': 0.0, 'neu': 0.488, 'pos': 0.512, 'compound': 0.484}

In [8]:
sia.polarity_scores("NLTK is bad!")

{'neg': 0.655, 'neu': 0.345, 'pos': 0.0, 'compound': -0.5848}

In [ ]:
df.iloc[[0,1,2,3]]

In [ ]:
datasent = df.apply(lambda x: sia.polarity_scores(x['Tweet Content']), 1)
df = df.join(pd.DataFrame(list(datasent)))
df

In [ ]:
df

In [10]:
df[['neg','neu','pos','compound']].sum()

neg          2699.5530
neu         28289.4240
pos          2184.1080
compound    -2187.4991
dtype: float64

In [20]:
bins = [0, 0.4, 0.50, 0.6]
names = ['lower', 'medium', 'high']
df['neg_category']  = pd.cut(df['neg'], bins, labels=names, include_lowest =True)
df['pos_category']  = pd.cut(df['pos'], bins, labels=names, include_lowest =True)


In [21]:
df[df['neg_category'] == 'high'].shape

(12, 29)

In [23]:
df[df['pos_category'] == 'high'].shape

(15, 29)

In [25]:
df['compound'].describe()

count    33174.000000
mean        -0.065940
std          0.461793
min         -0.982800
25%         -0.421500
50%          0.000000
75%          0.318200
max          0.986200
Name: compound, dtype: float64

In [27]:
bins = [-np.inf, -0.5, 0.5, 1]
names = ['high_negative', 'neu', 'high_positive']
df['compound_category']  = pd.cut(df['compound'], bins, labels=names, include_lowest =True)

In [34]:
df[df['compound_category'] == 'high_positive']

7        “When the virus hit, I felt that I should stay...
40       The closure of #Iran's parliament comes after ...
57       Join me on this webinar today. It's free and o...
64       How can public health advocates encourage citi...
67       Some beards are better then others if wearing ...
                               ...                        
33120    RT @hayesluk: Point 3 is especially insightful...
33124    RT @hayesluk: Point 3 is especially insightful...
33128    RT @hayesluk: Point 3 is especially insightful...
33129    RT @hayesluk: Point 3 is especially insightful...
33133    Point 3 is especially insightful after Prof. D...
Name: Tweet Content, Length: 4119, dtype: object

In [86]:
logical_vector = df['compound_category'] == 'high_positive'

In [90]:
sum(logical_vector)

4119

In [91]:
df_freq_target = pd.DataFrame(np.asarray(freq_term_DTM[logical_vector].sum(0).T).reshape(-1))

In [92]:
df_freq_target.index = [word for (word,idx) in sorted(vectorized.vocabulary_.items(), key= lambda x:x[1])]

In [93]:
df_freq_target.columns = ['freq1']
df_freq_target['freq2'] = np.asarray(freq_term_DTM[~(logical_vector)].sum(0).T).reshape(-1)

In [94]:
df_freq_target['tot'] = df_freq_target['freq1'] + df_freq_target['freq2']

In [95]:

df_freq_target['freq1'] = df_freq_target['freq1'].apply(lambda x: 0.0000001 if x == 0 else x).astype(float)
df_freq_target['freq2'] = df_freq_target['freq2'].apply(lambda x: 0.0000001 if x == 0 else x).astype(float)
#
df_freq_target['freq1_norm'] = df_freq_target['freq1']/df_freq_target['freq1'].sum() * 1000000
df_freq_target['freq2_norm'] = df_freq_target['freq2']/df_freq_target['freq2'].sum() * 1000000
#
df_freq_target['fraction'] = df_freq_target['freq1_norm'] / df_freq_target['freq2_norm']
df_freq_target['Log-likelihood Ratio'] = df_freq_target['fraction'].apply(math.log2)

In [96]:
import math
df_freq_target['Log-likelihood Ratio'] = df_freq_target['fraction'].apply(math.log2)

In [98]:
frequency_threshold = 10 # Insert your frequency threshold as integer
df_freq_target[df_freq_target['tot'] > frequency_threshold]['Log-likelihood Ratio'].sort_values(ascending=False).iloc[range(50)]

choculate             33.384676
relax                 32.079821
tornadotrump          31.773718
tombstonewyatt        31.773718
arveecasil09          31.484212
vid                   31.418623
retested              31.349911
followthemoney        31.079821
redistribution        31.079821
ahhh                  31.079821
reinforces            30.799713
linkupfr              30.799713
fwasteria             30.799713
atomaraullo           30.747246
stormsabine           30.692798
dgb                   30.692798
rajeev                30.692798
lipa                  30.692798
pennyzhou025          30.692798
linkup                30.692798
firefightaustralia    30.692798
tempeteciara          30.692798
fernando              30.692798
creative               8.466782
profound               8.323824
jason                  7.908787
tide                   7.693058
speaker                7.470666
dua                    7.439302
patrol                 7.270243
epoch                  6.908787
commitme

In [85]:
df_freq_target['freq1'].sort_values(ascending=False).iloc[range(20)]

rt           5232.0
china        2615.0
people       1928.0
wuhan        1168.0
doctor       1145.0
death        1063.0
health       1022.0
case         1007.0
city          936.0
ha            896.0
lady          883.0
infected      882.0
wa            867.0
patient       859.0
new           852.0
virus         760.0
confirmed     702.0
public        699.0
chinese       697.0
amp           667.0
Name: freq1, dtype: float64

In [58]:
df_freq_target

,freq1,freq2,tot,freq1_norm,freq2_norm,fraction,Log-likelihood Ratio
abc7,1.000000e-07,102.0,102,0.000002,303.936876,6.692071e-09,-27.154900
abc7newsbayarea,7.000000e+00,155.0,162,142.377708,461.864861,3.082670e-01,-1.697748
able,1.000000e+01,56.0,66,203.396725,166.867305,1.218913e+00,0.285595
abscbnnews,1.000000e-07,117.0,117,0.000002,348.633476,5.834113e-09,-27.352839
abuse,1.000000e-07,619.0,619,0.000002,1844.479672,1.102732e-09,-29.756270
...,...,...,...,...,...,...,...
york,4.000000e+00,55.0,59,81.358690,163.887531,4.964300e-01,-1.010338
youtube,1.500000e+01,119.0,134,305.095088,354.593023,8.604092e-01,-0.216905
zero,1.000000e-07,56.0,56,0.000002,166.867305,1.218913e-08,-26.289830
zerohedge,5.000000e+00,72.0,77,101.698363,214.543678,4.740217e-01,-1.076975


In [ ]:

def lexical_keyness(DTM, df, categories, vocabulary_vectorize, terget_category):
    import math
    # This code ref takes inspiration from this python module : https://pypi.org/project/corpus-toolkit/
    # and its main script:  https://github.com/kristopherkyle/corpus_toolkit/blob/master/corpus_toolkit/corpus_tools.py
    # which is based on this paper: https://aclanthology.org/J93-1003/

    cluster_keyness = n_cluster
    logical_vector = df['compound_category'] == 'high_positive'
    df_freq_target = pd.DataFrame(np.asarray(DTM[cls_kmeans.labels_ == cluster_keyness].sum(0).T).reshape(-1))#, columns = [word for (word,idx) in sorted(vectorized.vocabulary_.items(), key= lambda x:x[1])]))
    df_freq_target.index = [word for (word,idx) in sorted(vocabulary_vectorize.items(), key= lambda x:x[1])]
    df_freq_target.index
    df_freq_target.columns = ['freq1']
    df_freq_target['freq2'] = np.asarray(DTM[~(cls_kmeans.labels_ == cluster_keyness)].sum(0).T).reshape(-1)
    #
    df_freq_target['tot'] = df_freq_target['freq1'] + df_freq_target['freq2']
    #
    df_freq_target['freq1'] = df_freq_target['freq1'].apply(lambda x: 0.0000001 if x == 0 else x).astype(float)
    df_freq_target['freq2'] = df_freq_target['freq2'].apply(lambda x: 0.0000001 if x == 0 else x).astype(float)
    #
    df_freq_target['freq1_norm'] = df_freq_target['freq1']/df_freq_target['freq1'].sum() * 1000000
    df_freq_target['freq2_norm'] = df_freq_target['freq2']/df_freq_target['freq2'].sum() * 1000000
    #
    df_freq_target['fraction'] = df_freq_target['freq1_norm'] / df_freq_target['freq2_norm']
    df_freq_target['Log-likelihood Ratio'] = df_freq_target['fraction'].apply(math.log2)
    return df_freq_target


In [174]:
datasent

0    {'neg': 0.077, 'neu': 0.923, 'pos': 0.0, 'comp...
1    {'neg': 0.0, 'neu': 0.917, 'pos': 0.083, 'comp...
2    {'neg': 0.0, 'neu': 0.839, 'pos': 0.161, 'comp...
3    {'neg': 0.092, 'neu': 0.789, 'pos': 0.119, 'co...
dtype: object